In [1]:
import pandas as pd
import os 
import numpy as np
import math

# Choix de la Saison

## Récupération de la liste des matchs

In [2]:
# Chemin vers le dossier de base
chemin_dossier = "data/matches"

# Liste pour stocker les DataFrames de chaque fichier JSON
dataframes = []

# Parcourir tous les fichiers dans le répertoire et ses sous-répertoires
for root, dirs, files in os.walk(chemin_dossier):
    for fichier in files:
        if fichier.endswith(".json"):
            chemin_fichier = os.path.join(root, fichier)
            
            # Lire le fichier JSON et créer un DataFrame
            df = pd.read_json(chemin_fichier)
            
            # Ajouter le DataFrame à la liste
            dataframes.append(df)

# Concaténer tous les DataFrames en un seul
df_matches = pd.concat(dataframes, ignore_index=True)

In [3]:
# Récuération des informations pertinentes dans les nested DataFrames
df_matches['competition'] = df_matches['competition'].apply(lambda x: x['country_name'])
df_matches['season'] = df_matches['season'].apply(lambda x: x['season_name'])
df_matches['home_team'] = df_matches['home_team'].apply(lambda x: x['home_team_name'])
df_matches['away_team'] = df_matches['away_team'].apply(lambda x: x['away_team_name'])

## Afficher les ligues avec le plus de matchs

In [4]:
df_matches.groupby('competition').size()

competition
Africa                        52
Argentina                      2
England                      744
Europe                       103
France                       435
Germany                      340
India                        115
International                264
Italy                        381
North and Central America      1
Spain                        871
United States of America      42
dtype: int64

## Choix de la Saison de Liga 

In [5]:
df_matches[df_matches['competition'] == 'Spain'].groupby('season').size()

season
1973/1974      1
1977/1978      1
1982/1983      1
1983/1984      1
2004/2005      7
2005/2006     17
2006/2007     26
2007/2008     28
2008/2009     31
2009/2010     35
2010/2011     33
2011/2012     37
2012/2013     32
2013/2014     31
2014/2015     38
2015/2016    380
2016/2017     34
2017/2018     36
2018/2019     34
2019/2020     33
2020/2021     35
dtype: int64

## Récupération de la liste des matchs de la Saison 2015/2016, seule saison complète

In [6]:
df_spain = df_matches[(df_matches['competition'] == 'Spain') & (df_matches['season'] == '2015/2016')]
spain_matches_list = df_spain['match_id'].tolist() 

## Récupération de l'ensembles des évènements de cette saison

On va créer un dictionnaire de DataFrames, en séparant les évènements par type d'évènement, afin de faciliter les manipulations futures et d'éviter la manipulation d'un DataFrame trop lourd

Manipulation qui prend plusieurs minutes

In [7]:
chemin_dossier = "data/events"

# Dictionnaire pour stocker les DataFrames par type
dfs_by_type_global = {}

# Parcourir tous les fichiers dans le répertoire
for fichier in os.listdir(chemin_dossier):
    # Enlever l'extension .json pour comparer avec la liste
    nom_fichier_sans_extension = int(os.path.splitext(fichier)[0])

    if nom_fichier_sans_extension in spain_matches_list:
        chemin_fichier = os.path.join(chemin_dossier, fichier)
        
        # Lire le fichier JSON et créer un DataFrame
        df = pd.read_json(chemin_fichier)
        # Nettoyage du DataFrame
        df.drop(['id', 'period', 'index','timestamp', 'minute', 'second', 'possession', 'possession_team', 'related_events', 'duration', 'tactics', 'team', 'off_camera'], axis=1, inplace=True)
        df.dropna(subset=['player'], inplace=True)
        df['type'] = df['type'].apply(lambda x: x['name']).str.replace('*', '')
        df['play_pattern'] = df['play_pattern'].apply(lambda x: x['name'])
        df['player'] = df['player'].apply(lambda x: x['name'])
        df['position'] = df['position'].apply(lambda x: x['name'])

        dfs_by_type_local = {}

        # Parcourir chaque type unique dans la colonne 'type'
        for type_value in df['type'].unique():
            # Créer un DataFrame pour chaque type
            dfs_by_type_local[type_value] = df[df['type'] == type_value].dropna(axis=1, how='all')

            columns = dfs_by_type_local[type_value].columns
            # Décompression des nested Dataframes
            for col in columns:
                # Vérifier si la colonne contient des dictionnaires
                if isinstance(dfs_by_type_local[type_value][col].iloc[0], dict):
                    # Utiliser pd.json_normalize pour décompresser la colonne
                    nested_df = pd.json_normalize(dfs_by_type_local[type_value][col])
                    # Supprimer la colonne d'origine
                    dfs_by_type_local[type_value].drop(col, axis=1, inplace=True)
                    # Ajouter les colonnes décompressées au DataFrame
                    dfs_by_type_local[type_value] = dfs_by_type_local[type_value].reset_index(drop=True)
                    nested_df = nested_df.reset_index(drop=True)
                    dfs_by_type_local[type_value] = pd.concat([dfs_by_type_local[type_value], nested_df], axis=1)
        
        # Ajouter les DataFrames locaux au dictionnaire global
        for type_value, df_local in dfs_by_type_local.items():
            if type_value not in dfs_by_type_global:
                dfs_by_type_global[type_value] = []
            dfs_by_type_global[type_value].append(df_local)

# Concaténer tous les DataFrames de chaque type
dfs_by_type_final = {}
for type_value, dfs_list in dfs_by_type_global.items():
    dfs_by_type_final[type_value] = pd.concat(dfs_list, ignore_index=True)


## Récupération des informations générales sur chaque joueur à chaque match (titularisation, temps de jeu, position etc.)

In [8]:
chemin_dossier = "data/lineups"

# Liste pour stocker les DataFrames de chaque fichier JSON
dataframes = []

for fichier in os.listdir(chemin_dossier):
    # Enlever l'extension .json pour comparer avec la liste
    nom_fichier_sans_extension = int(os.path.splitext(fichier)[0])

    if nom_fichier_sans_extension in spain_matches_list:
        chemin_fichier = os.path.join(chemin_dossier, fichier)
        
        # Lire le fichier JSON et créer un DataFrame
        df = pd.read_json(chemin_fichier)
        
        # Ajouter une colonne match_id avec le nom du fichier sans extension
        df['match_id'] = nom_fichier_sans_extension
        
        # Ajouter le DataFrame à la liste
        dataframes.append(df)

# Concaténer tous les DataFrames en un seul
df_lineups = pd.concat(dataframes, ignore_index=True)

# Décompression des nested DataFrames
df_lineups = df_lineups.explode('lineup').reset_index(drop=True)
df_lineups = pd.concat([df_lineups['match_id'], df_lineups['team_name'], pd.json_normalize(df_lineups['lineup'])], axis = 1)
df_lineups = df_lineups.explode('positions').reset_index(drop=True).drop(['player_id', 'player_nickname', 'cards', 'country.id'], axis=1)

In [9]:
# Fonction pour convertir les temps de début et de fin en minutes
def convert_to_minutes(time_str):
    try:
        minutes, seconds = map(int, time_str.split(':'))
        return minutes + seconds / 60
    except (ValueError, AttributeError):
        return None

# Fonction pour récupérer la position
def get_position(pos):
    try:
        return pos['position']
    except (TypeError, KeyError):
        return None
    
# Fonction pour récupérer le temps de jeu
def get_time(pos):
    try:
        from_time = convert_to_minutes(pos['from'])
        to_time = convert_to_minutes(pos['to'] if pos['to'] is not None else '92:00')
        return to_time - from_time
    except (TypeError, KeyError, ValueError):
        return None

# Fonction pour déterminer si le joueur est titulaire
def is_starting(pos):
    try:
        return pos['start_reason'] == "Starting XI"
    except (TypeError, KeyError):
        return False
    
# Fonction pour déterminer si le joueur est remplaçant
def is_subbing(pos):
    try:
        return pos['start_reason'].startswith("Substitution - On")
    except (TypeError, KeyError):
        return False

# Application des fonctions
df_lineups['position'] = df_lineups['positions'].apply(get_position)
df_lineups['time'] = df_lineups['positions'].apply(get_time)
df_lineups['starting'] = df_lineups['positions'].apply(is_starting)
df_lineups['subbing'] = df_lineups['positions'].apply(is_subbing)   
df_lineups.drop('positions', axis=1, inplace=True)
df_lineups

,match_id,team_name,player_name,jersey_number,country.name,position,time,starting,subbing
0,265839,Sevilla,Adil Rami,3,France,Right Center Back,92.000000,True,False
1,265839,Sevilla,Vicente Iborra De La Fuente,8,Spain,Center Attacking Midfield,74.250000,True,False
2,265839,Sevilla,Steven N''Kemboanza Mike Christopher Nzonzi,15,France,Left Center Midfield,92.000000,True,False
3,265839,Sevilla,Michael Krohn-Dehli,7,Denmark,Left Wing,63.733333,True,False
4,265839,Sevilla,Michael Krohn-Dehli,7,Denmark,Right Center Midfield,28.266667,False,False
...,...,...,...,...,...,...,...,...,...
16729,3825908,Eibar,Saúl Berjón Pérez,21,Spain,None,NaN,False,False
16730,3825908,Eibar,Borja Ekiza Imaz,23,Spain,Right Back,45.000000,True,False
16731,3825908,Eibar,Jon Ansotegi Gorostola,2,Spain,Left Center Back,92.000000,True,False
16732,3825908,Eibar,Aleksandar Pantić,3,Serbia,None,NaN,False,False


## Récupération des résultats de chaque match

In [10]:
chemin_fichier = "data/matches/11/27.json"

# Lire le fichier JSON et créer un DataFrame
df_results = pd.read_json(chemin_fichier)

# Nettoyage du DataFrame
df_results['home_team'] = df_results['home_team'].apply(lambda x: x['home_team_name'])
df_results['away_team'] = df_results['away_team'].apply(lambda x: x['away_team_name'])
df_results = df_results[['match_id', 'home_team', 'away_team', 'home_score', 'away_score']]

# Ajouter une colonne 'result' pour déterminer le résultat du match
df_results['result'] = df_results.apply(lambda x: x['home_team'] if x['home_score'] > x['away_score'] else x['away_team'] if x['away_score'] > x['home_score'] else 'Draw', axis=1)

df_results = df_results.drop(['home_team', 'away_team', 'home_score', 'away_score'], axis=1)

# Fusionner les DataFrames
df_players = pd.merge(df_lineups, df_results, on='match_id', how='left')
df_players['win'] = df_players['team_name'] == df_players['result']
df_players['draw'] = df_players['result'] == 'Draw'
df_players['lose'] = ~(df_players['win'] | df_players['draw'])
df_players = df_players.drop('result', axis=1)

df_players = df_players.dropna(subset=['position'], axis=0)
# Pour les joueurs ayant changé de poste au cours du match, on change les bouléens de résultat pour ne pas les compter deux fois
condition = (df_players['starting'] == False) & (df_players['subbing'] == False)
df_players.loc[condition, ['win', 'draw', 'lose']] = False

df_players

,match_id,team_name,player_name,jersey_number,country.name,position,time,starting,subbing,win,draw,lose
0,265839,Sevilla,Adil Rami,3,France,Right Center Back,92.000000,True,False,False,False,True
1,265839,Sevilla,Vicente Iborra De La Fuente,8,Spain,Center Attacking Midfield,74.250000,True,False,False,False,True
2,265839,Sevilla,Steven N''Kemboanza Mike Christopher Nzonzi,15,France,Left Center Midfield,92.000000,True,False,False,False,True
3,265839,Sevilla,Michael Krohn-Dehli,7,Denmark,Left Wing,63.733333,True,False,False,False,True
4,265839,Sevilla,Michael Krohn-Dehli,7,Denmark,Right Center Midfield,28.266667,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
16725,3825908,Eibar,David Juncà Reñé,17,Spain,Left Back,16.350000,False,False,False,False,False
16726,3825908,Eibar,Mauro Javier Dos Santos,15,Argentina,Right Center Back,92.000000,True,False,False,False,True
16730,3825908,Eibar,Borja Ekiza Imaz,23,Spain,Right Back,45.000000,True,False,False,False,True
16731,3825908,Eibar,Jon Ansotegi Gorostola,2,Spain,Left Center Back,92.000000,True,False,False,False,True


## Préparation des statistiques pertinentes sur chaque DataFrame

## Agrégation du DataFrame df_players par joueur

In [11]:
columns_to_sum = ['time', 'starting', 'subbing', 'win', 'draw', 'lose']  
columns_to_mode = ['team_name', 'jersey_number', 'country.name', 'position']

def get_mode(series):
    if series.mode().empty:
        return np.nan  
    else:
        return series.mode().iloc[0]

agg_dict = {col: get_mode for col in columns_to_mode}
agg_dict.update({col: 'sum' for col in columns_to_sum})


df_players = df_players.groupby('player_name', as_index=False).agg(agg_dict)

# On enlève les joueurs ayant joués moins de 500 minutes, car statistiques non significatives
df_players = df_players[df_players['time'] > 500]

## Passes

In [12]:
# Nettoyage du DataFrame
df_all_pass = dfs_by_type_final['Pass'].drop(['counterpress', 'angle', 'recipient.id', 'height.id', 'body_part.id', 'type.id', 'outcome.id', 'technique.id', 'assisted_shot_id'], axis=1)
df_all_pass[['goal_assist', 'shot_assist', 'cross']]= df_all_pass[['goal_assist', 'shot_assist', 'cross']].fillna(False)
# Création de nouvelles colonnes pour le type de passe
df_all_pass['long_pass'] = np.where(df_all_pass['length'] > 40, 1, 0)
df_all_pass['medium_pass'] = np.where((df_all_pass['length'] <= 40) & (df_all_pass['length'] > 20), 1, 0)
df_all_pass['short_pass'] = np.where(df_all_pass['length'] <= 20, 1, 0)

# Création d'un DataFrame pour les passes réussies
df_pass = df_all_pass[df_all_pass['outcome.name'].isna()]

# Création du DataFrame agrégé par joueur
df_stats_final = df_pass.groupby('player').size().reset_index(name='complete_pass').merge(df_pass.groupby('player')['long_pass'].sum().reset_index(name='long_pass'), 
        on='player').merge(df_pass.groupby('player')['medium_pass'].sum().reset_index(name='medium_pass'), on='player').merge(df_pass.groupby('player')['short_pass'].sum().reset_index(name='short_pass'), 
        on='player').merge(df_pass.groupby('player')['goal_assist'].sum().reset_index(name='goal_assist'), on='player').merge(df_all_pass.groupby('player').size().reset_index(name='all_pass'),
        on='player').merge(df_all_pass.groupby('player')['shot_assist'].sum().reset_index(name='shot_assist'), on='player').merge(df_all_pass.groupby('player')['cross'].sum().reset_index(name='cross'),
        on='player')


df_stats_final = df_players.merge(df_stats_final, left_on='player_name', right_on='player').drop('player', axis=1)

pass_columns = ['complete_pass', 'long_pass', 'medium_pass', 'goal_assist', 'short_pass', 'shot_assist', 'cross']

# Calculer les passes par 90 minutes pour chaque colonne de la liste
for col in pass_columns:
    df_stats_final[f'{col}_per_90_min'] = np.round(df_stats_final[col] * 90 / df_stats_final['time'], 2)

# Calculer le taux de réussite des passes
df_stats_final['pass_success_rate'] = np.round(df_stats_final['complete_pass'] / df_stats_final['all_pass'] * 100, 2)


df_stats_final


C:\Users\vtgra\AppData\Local\Temp\ipykernel_23928\3167440179.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_all_pass[['goal_assist', 'shot_assist', 'cross']]= df_all_pass[['goal_assist', 'shot_assist', 'cross']].fillna(False)


,player_name,team_name,jersey_number,country.name,position,time,starting,subbing,win,draw,...,shot_assist,cross,complete_pass_per_90_min,long_pass_per_90_min,medium_pass_per_90_min,goal_assist_per_90_min,short_pass_per_90_min,shot_assist_per_90_min,cross_per_90_min,pass_success_rate
0,Abdoul Karim Yoda,Getafe,11,France,Left Back,1009.400000,8,10,2,6,...,10,27,34.95,1.87,7.85,0.00,25.23,0.89,2.41,74.52
1,Abdoulaye Doucouré,Granada,16,Mali,Left Defensive Midfield,1184.450000,13,2,5,4,...,9,3,36.62,1.90,10.18,0.15,24.54,0.68,0.23,83.54
2,Abraham González Casanova,Espanyol,10,Spain,Left Center Midfield,1128.083333,13,7,5,5,...,15,12,33.03,1.83,10.77,0.08,20.42,1.20,0.96,77.97
3,Adalberto Peñaranda Maestre,Granada,27,Venezuela (Bolivarian Republic),Right Center Forward,1694.933333,21,2,8,3,...,12,17,14.66,0.32,2.76,0.16,11.58,0.64,0.90,72.44
4,Aderllan Leandro de Jesus Santos,Valencia,4,Brazil,Left Center Back,1564.000000,17,0,5,3,...,1,0,31.36,1.55,15.19,0.00,14.62,0.06,0.00,82.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Íñigo Lekue Martínez,Athletic Club,30,Spain,Left Wing,1187.166667,12,8,10,5,...,11,43,33.43,1.36,7.28,0.00,24.79,0.83,3.26,74.24
410,Íñigo Martínez Berridi,Real Sociedad,6,Spain,Left Center Back,2760.000000,30,0,9,9,...,2,1,37.01,3.03,15.68,0.00,18.29,0.07,0.03,78.11
411,Óliver Torres Muñoz,Atlético Madrid,10,Spain,Right Midfield,838.916667,9,12,17,2,...,13,8,44.41,1.93,9.55,0.21,32.94,1.39,0.86,81.02
412,Óscar Esau Duarte Gaitán,Espanyol,24,Costa Rica,Left Center Back,1220.716667,13,1,6,3,...,1,1,25.73,2.36,8.85,0.07,14.52,0.07,0.07,74.57


## Ball Receipt 

In [13]:
# Renseigne seulement sur le nombre de ballons reçus et les ballons perdus
df_ball_receipt = dfs_by_type_final['Ball Receipt']

# Fonction pour extraire l'info de la perte de balle
def extract_outcome_name(ball_receipt):
    if isinstance(ball_receipt, dict) and 'outcome' in ball_receipt:
        return True
    return False

# Appliquer la fonction à la colonne 'ball_receipt'
df_ball_receipt['lost_ball'] = df_ball_receipt['ball_receipt'].apply(extract_outcome_name)

# Agrégation nombre de ballons touchés par joueur
df_ball_receipt_total = df_ball_receipt.groupby('player').size().reset_index(name='ball_receipt')

# Agrégation nombre de ballons perdus par joueur
df_lost_ball = df_ball_receipt.groupby('player')['lost_ball'].sum()

df_stats_final = df_stats_final.merge(df_ball_receipt_total, left_on='player_name', right_on='player').merge(df_lost_ball, left_on='player_name', right_on='player').drop('player', axis=1)

# Création Statistiques d'intérêt
df_stats_final['ball_receipt_per_90_min'] = np.round(df_stats_final['ball_receipt'] * 90 / df_stats_final['time'], 2)
df_stats_final['lost_ball_per_90_min'] = np.round(df_stats_final['lost_ball'] * 90 / df_stats_final['time'], 2)
df_stats_final

,player_name,team_name,jersey_number,country.name,position,time,starting,subbing,win,draw,...,medium_pass_per_90_min,goal_assist_per_90_min,short_pass_per_90_min,shot_assist_per_90_min,cross_per_90_min,pass_success_rate,ball_receipt,lost_ball,ball_receipt_per_90_min,lost_ball_per_90_min
0,Abdoul Karim Yoda,Getafe,11,France,Left Back,1009.400000,8,10,2,6,...,7.85,0.00,25.23,0.89,2.41,74.52,443,46,39.50,4.10
1,Abdoulaye Doucouré,Granada,16,Mali,Left Defensive Midfield,1184.450000,13,2,5,4,...,10.18,0.15,24.54,0.68,0.23,83.54,483,49,36.70,3.72
2,Abraham González Casanova,Espanyol,10,Spain,Left Center Midfield,1128.083333,13,7,5,5,...,10.77,0.08,20.42,1.20,0.96,77.97,505,93,40.29,7.42
3,Adalberto Peñaranda Maestre,Granada,27,Venezuela (Bolivarian Republic),Right Center Forward,1694.933333,21,2,8,3,...,2.76,0.16,11.58,0.64,0.90,72.44,766,299,40.67,15.88
4,Aderllan Leandro de Jesus Santos,Valencia,4,Brazil,Left Center Back,1564.000000,17,0,5,3,...,15.19,0.00,14.62,0.06,0.00,82.95,486,5,27.97,0.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Íñigo Lekue Martínez,Athletic Club,30,Spain,Left Wing,1187.166667,12,8,10,5,...,7.28,0.00,24.79,0.83,3.26,74.24,496,63,37.60,4.78
410,Íñigo Martínez Berridi,Real Sociedad,6,Spain,Left Center Back,2760.000000,30,0,9,9,...,15.68,0.00,18.29,0.07,0.03,78.11,995,27,32.45,0.88
411,Óliver Torres Muñoz,Atlético Madrid,10,Spain,Right Midfield,838.916667,9,12,17,2,...,9.55,0.21,32.94,1.39,0.86,81.02,552,71,59.22,7.62
412,Óscar Esau Duarte Gaitán,Espanyol,24,Costa Rica,Left Center Back,1220.716667,13,1,6,3,...,8.85,0.07,14.52,0.07,0.07,74.57,276,18,20.35,1.33


## Own Goal 

In [14]:
# On a deux DataFrames qui désignent la même chose, on les fusionne
dfs_by_type_final['Own Goal'] = pd.concat([dfs_by_type_final['Own Goal Against'], dfs_by_type_final['Own Goal For']], ignore_index=True)
dfs_by_type_final['Own Goal']['type'] = 'Own Goal'
df_own_goal = dfs_by_type_final['Own Goal'].groupby('player').size().reset_index(name='own_goal')
df_own_goal = df_own_goal.replace(0, np.nan)

# On ajoute aux statistiques finales
df_stats_final = df_stats_final.merge(df_own_goal, left_on='player_name', right_on='player', how='left').drop('player', axis=1)
df_stats_final['own_goal'] = df_stats_final['own_goal'].fillna(0)
df_stats_final

,player_name,team_name,jersey_number,country.name,position,time,starting,subbing,win,draw,...,goal_assist_per_90_min,short_pass_per_90_min,shot_assist_per_90_min,cross_per_90_min,pass_success_rate,ball_receipt,lost_ball,ball_receipt_per_90_min,lost_ball_per_90_min,own_goal
0,Abdoul Karim Yoda,Getafe,11,France,Left Back,1009.400000,8,10,2,6,...,0.00,25.23,0.89,2.41,74.52,443,46,39.50,4.10,0.0
1,Abdoulaye Doucouré,Granada,16,Mali,Left Defensive Midfield,1184.450000,13,2,5,4,...,0.15,24.54,0.68,0.23,83.54,483,49,36.70,3.72,0.0
2,Abraham González Casanova,Espanyol,10,Spain,Left Center Midfield,1128.083333,13,7,5,5,...,0.08,20.42,1.20,0.96,77.97,505,93,40.29,7.42,0.0
3,Adalberto Peñaranda Maestre,Granada,27,Venezuela (Bolivarian Republic),Right Center Forward,1694.933333,21,2,8,3,...,0.16,11.58,0.64,0.90,72.44,766,299,40.67,15.88,0.0
4,Aderllan Leandro de Jesus Santos,Valencia,4,Brazil,Left Center Back,1564.000000,17,0,5,3,...,0.00,14.62,0.06,0.00,82.95,486,5,27.97,0.29,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Íñigo Lekue Martínez,Athletic Club,30,Spain,Left Wing,1187.166667,12,8,10,5,...,0.00,24.79,0.83,3.26,74.24,496,63,37.60,4.78,0.0
410,Íñigo Martínez Berridi,Real Sociedad,6,Spain,Left Center Back,2760.000000,30,0,9,9,...,0.00,18.29,0.07,0.03,78.11,995,27,32.45,0.88,0.0
411,Óliver Torres Muñoz,Atlético Madrid,10,Spain,Right Midfield,838.916667,9,12,17,2,...,0.21,32.94,1.39,0.86,81.02,552,71,59.22,7.62,0.0
412,Óscar Esau Duarte Gaitán,Espanyol,24,Costa Rica,Left Center Back,1220.716667,13,1,6,3,...,0.07,14.52,0.07,0.07,74.57,276,18,20.35,1.33,1.0


## Dispossessed

In [15]:
# S'ajoute aux ballons perdus
df_dispossessed = dfs_by_type_final['Dispossessed']
df_dispossessed = df_dispossessed.groupby('player').size().reset_index(name='dispossessed')

df_stats_final = df_stats_final.merge(df_dispossessed, left_on='player_name', right_on='player', how='left').drop('player', axis=1)

df_stats_final['lost_ball'] = df_stats_final['lost_ball'] + df_stats_final['dispossessed']
df_stats_final['lost_ball_per_90_min'] = np.round(df_stats_final['lost_ball'] * 90 / df_stats_final['time'], 2)
df_stats_final = df_stats_final.drop('dispossessed', axis=1)
df_stats_final

,player_name,team_name,jersey_number,country.name,position,time,starting,subbing,win,draw,...,goal_assist_per_90_min,short_pass_per_90_min,shot_assist_per_90_min,cross_per_90_min,pass_success_rate,ball_receipt,lost_ball,ball_receipt_per_90_min,lost_ball_per_90_min,own_goal
0,Abdoul Karim Yoda,Getafe,11,France,Left Back,1009.400000,8,10,2,6,...,0.00,25.23,0.89,2.41,74.52,443,64.0,39.50,5.71,0.0
1,Abdoulaye Doucouré,Granada,16,Mali,Left Defensive Midfield,1184.450000,13,2,5,4,...,0.15,24.54,0.68,0.23,83.54,483,64.0,36.70,4.86,0.0
2,Abraham González Casanova,Espanyol,10,Spain,Left Center Midfield,1128.083333,13,7,5,5,...,0.08,20.42,1.20,0.96,77.97,505,100.0,40.29,7.98,0.0
3,Adalberto Peñaranda Maestre,Granada,27,Venezuela (Bolivarian Republic),Right Center Forward,1694.933333,21,2,8,3,...,0.16,11.58,0.64,0.90,72.44,766,362.0,40.67,19.22,0.0
4,Aderllan Leandro de Jesus Santos,Valencia,4,Brazil,Left Center Back,1564.000000,17,0,5,3,...,0.00,14.62,0.06,0.00,82.95,486,9.0,27.97,0.52,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Íñigo Lekue Martínez,Athletic Club,30,Spain,Left Wing,1187.166667,12,8,10,5,...,0.00,24.79,0.83,3.26,74.24,496,83.0,37.60,6.29,0.0
410,Íñigo Martínez Berridi,Real Sociedad,6,Spain,Left Center Back,2760.000000,30,0,9,9,...,0.00,18.29,0.07,0.03,78.11,995,38.0,32.45,1.24,0.0
411,Óliver Torres Muñoz,Atlético Madrid,10,Spain,Right Midfield,838.916667,9,12,17,2,...,0.21,32.94,1.39,0.86,81.02,552,97.0,59.22,10.41,0.0
412,Óscar Esau Duarte Gaitán,Espanyol,24,Costa Rica,Left Center Back,1220.716667,13,1,6,3,...,0.07,14.52,0.07,0.07,74.57,276,22.0,20.35,1.62,1.0


## Clearance 

In [16]:
df_clearance = dfs_by_type_final['Clearance'].drop(['body_part.id', 'body_part.name'], axis=1)
df_clearance = df_clearance.fillna(False)

df_clearance = df_clearance.groupby('player').agg(
    total_clearance=('player', 'size'),
    left_foot=('left_foot', 'sum'),
    right_foot=('right_foot', 'sum'),
    head=('head', 'sum'),
    other=('other', 'sum'),
    aerial_won=('aerial_won', 'sum')
)

df_clearance = df_clearance.rename(columns={
    'left_foot': 'left_foot_clearance',
    'right_foot': 'right_foot_clearance',
    'head': 'head_clearance',
    'other': 'other_clearance',
    'aerial_won': 'aerial_won_clearance'
})

df_stats_final = df_stats_final.merge(df_clearance, left_on='player_name', right_on='player', how='left')
df_stats_final['clearance_per_90_min'] = np.round(df_stats_final['total_clearance'] * 90 / df_stats_final['time'], 2)
df_stats_final['aerial_duel_won_per_90_min'] = np.round(df_stats_final['aerial_won_clearance'] * 90 / df_stats_final['time'], 2)
df_stats_final

C:\Users\vtgra\AppData\Local\Temp\ipykernel_23928\1671202383.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clearance = df_clearance.fillna(False)


,player_name,team_name,jersey_number,country.name,position,time,starting,subbing,win,draw,...,lost_ball_per_90_min,own_goal,total_clearance,left_foot_clearance,right_foot_clearance,head_clearance,other_clearance,aerial_won_clearance,clearance_per_90_min,aerial_duel_won_per_90_min
0,Abdoul Karim Yoda,Getafe,11,France,Left Back,1009.400000,8,10,2,6,...,5.71,0.0,20.0,3.0,3.0,13.0,1.0,3.0,1.78,0.27
1,Abdoulaye Doucouré,Granada,16,Mali,Left Defensive Midfield,1184.450000,13,2,5,4,...,4.86,0.0,15.0,3.0,8.0,4.0,0.0,3.0,1.14,0.23
2,Abraham González Casanova,Espanyol,10,Spain,Left Center Midfield,1128.083333,13,7,5,5,...,7.98,0.0,7.0,2.0,3.0,2.0,0.0,1.0,0.56,0.08
3,Adalberto Peñaranda Maestre,Granada,27,Venezuela (Bolivarian Republic),Right Center Forward,1694.933333,21,2,8,3,...,19.22,0.0,5.0,0.0,4.0,1.0,0.0,0.0,0.27,0.00
4,Aderllan Leandro de Jesus Santos,Valencia,4,Brazil,Left Center Back,1564.000000,17,0,5,3,...,0.52,0.0,90.0,9.0,31.0,49.0,1.0,20.0,5.18,1.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Íñigo Lekue Martínez,Athletic Club,30,Spain,Left Wing,1187.166667,12,8,10,5,...,6.29,0.0,13.0,2.0,3.0,8.0,0.0,6.0,0.99,0.45
410,Íñigo Martínez Berridi,Real Sociedad,6,Spain,Left Center Back,2760.000000,30,0,9,9,...,1.24,0.0,130.0,55.0,20.0,55.0,0.0,26.0,4.24,0.85
411,Óliver Torres Muñoz,Atlético Madrid,10,Spain,Right Midfield,838.916667,9,12,17,2,...,10.41,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.21,0.00
412,Óscar Esau Duarte Gaitán,Espanyol,24,Costa Rica,Left Center Back,1220.716667,13,1,6,3,...,1.62,1.0,82.0,19.0,17.0,46.0,0.0,22.0,6.05,1.62


## Interception

In [17]:
df_interception = dfs_by_type_final['Interception'].drop(['counterpress', 'outcome.id'], axis=1)
df_interception = df_interception[df_interception['outcome.name'].isin(['Success In Play', 'Won', 'Success Out', 'Success'])]

df_interception = df_interception.groupby('player').size().reset_index(name='interception')

df_stats_final = df_stats_final.merge(df_interception, left_on='player_name', right_on='player', how='left').drop('player', axis=1)
df_stats_final['interception_per_90_min'] = np.round(df_stats_final['interception'] * 90 / df_stats_final['time'], 2)
df_stats_final

,player_name,team_name,jersey_number,country.name,position,time,starting,subbing,win,draw,...,total_clearance,left_foot_clearance,right_foot_clearance,head_clearance,other_clearance,aerial_won_clearance,clearance_per_90_min,aerial_duel_won_per_90_min,interception,interception_per_90_min
0,Abdoul Karim Yoda,Getafe,11,France,Left Back,1009.400000,8,10,2,6,...,20.0,3.0,3.0,13.0,1.0,3.0,1.78,0.27,17.0,1.52
1,Abdoulaye Doucouré,Granada,16,Mali,Left Defensive Midfield,1184.450000,13,2,5,4,...,15.0,3.0,8.0,4.0,0.0,3.0,1.14,0.23,17.0,1.29
2,Abraham González Casanova,Espanyol,10,Spain,Left Center Midfield,1128.083333,13,7,5,5,...,7.0,2.0,3.0,2.0,0.0,1.0,0.56,0.08,11.0,0.88
3,Adalberto Peñaranda Maestre,Granada,27,Venezuela (Bolivarian Republic),Right Center Forward,1694.933333,21,2,8,3,...,5.0,0.0,4.0,1.0,0.0,0.0,0.27,0.00,7.0,0.37
4,Aderllan Leandro de Jesus Santos,Valencia,4,Brazil,Left Center Back,1564.000000,17,0,5,3,...,90.0,9.0,31.0,49.0,1.0,20.0,5.18,1.15,25.0,1.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Íñigo Lekue Martínez,Athletic Club,30,Spain,Left Wing,1187.166667,12,8,10,5,...,13.0,2.0,3.0,8.0,0.0,6.0,0.99,0.45,15.0,1.14
410,Íñigo Martínez Berridi,Real Sociedad,6,Spain,Left Center Back,2760.000000,30,0,9,9,...,130.0,55.0,20.0,55.0,0.0,26.0,4.24,0.85,39.0,1.27
411,Óliver Torres Muñoz,Atlético Madrid,10,Spain,Right Midfield,838.916667,9,12,17,2,...,2.0,1.0,1.0,0.0,0.0,0.0,0.21,0.00,6.0,0.64
412,Óscar Esau Duarte Gaitán,Espanyol,24,Costa Rica,Left Center Back,1220.716667,13,1,6,3,...,82.0,19.0,17.0,46.0,0.0,22.0,6.05,1.62,15.0,1.11


## Foul won 

In [18]:
df_foul_won = dfs_by_type_final['Foul Won']
nested_df = pd.json_normalize(df_foul_won['foul_won'])
nested_df = nested_df.rename(columns={'penalty': 'penalty2',})
df_foul_won = pd.concat([df_foul_won[['player', 'penalty']], nested_df], axis=1)
df_foul_won['penalty'] = df_foul_won['penalty'].combine_first(df_foul_won['penalty2']).fillna(False)

df_foul_won = df_foul_won.groupby('player').agg(
    foul_won=('player', 'size'),
    penalty_won=('penalty', 'sum')
)

df_stats_final = df_stats_final.merge(df_foul_won, left_on='player_name', right_on='player', how='left')
df_stats_final['foul_won_per_90_min'] = np.round(df_stats_final['foul_won'] * 90 / df_stats_final['time'], 2)
df_stats_final


C:\Users\vtgra\AppData\Local\Temp\ipykernel_23928\1188980942.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_foul_won['penalty'] = df_foul_won['penalty'].combine_first(df_foul_won['penalty2']).fillna(False)


,player_name,team_name,jersey_number,country.name,position,time,starting,subbing,win,draw,...,head_clearance,other_clearance,aerial_won_clearance,clearance_per_90_min,aerial_duel_won_per_90_min,interception,interception_per_90_min,foul_won,penalty_won,foul_won_per_90_min
0,Abdoul Karim Yoda,Getafe,11,France,Left Back,1009.400000,8,10,2,6,...,13.0,1.0,3.0,1.78,0.27,17.0,1.52,10.0,0.0,0.89
1,Abdoulaye Doucouré,Granada,16,Mali,Left Defensive Midfield,1184.450000,13,2,5,4,...,4.0,0.0,3.0,1.14,0.23,17.0,1.29,8.0,0.0,0.61
2,Abraham González Casanova,Espanyol,10,Spain,Left Center Midfield,1128.083333,13,7,5,5,...,2.0,0.0,1.0,0.56,0.08,11.0,0.88,9.0,0.0,0.72
3,Adalberto Peñaranda Maestre,Granada,27,Venezuela (Bolivarian Republic),Right Center Forward,1694.933333,21,2,8,3,...,1.0,0.0,0.0,0.27,0.00,7.0,0.37,41.0,1.0,2.18
4,Aderllan Leandro de Jesus Santos,Valencia,4,Brazil,Left Center Back,1564.000000,17,0,5,3,...,49.0,1.0,20.0,5.18,1.15,25.0,1.44,6.0,0.0,0.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Íñigo Lekue Martínez,Athletic Club,30,Spain,Left Wing,1187.166667,12,8,10,5,...,8.0,0.0,6.0,0.99,0.45,15.0,1.14,10.0,0.0,0.76
410,Íñigo Martínez Berridi,Real Sociedad,6,Spain,Left Center Back,2760.000000,30,0,9,9,...,55.0,0.0,26.0,4.24,0.85,39.0,1.27,27.0,0.0,0.88
411,Óliver Torres Muñoz,Atlético Madrid,10,Spain,Right Midfield,838.916667,9,12,17,2,...,0.0,0.0,0.0,0.21,0.00,6.0,0.64,17.0,0.0,1.82
412,Óscar Esau Duarte Gaitán,Espanyol,24,Costa Rica,Left Center Back,1220.716667,13,1,6,3,...,46.0,0.0,22.0,6.05,1.62,15.0,1.11,5.0,0.0,0.37


## Foul commited

In [19]:
df_foul_committed = dfs_by_type_final['Foul Committed']
nested_df = pd.json_normalize(df_foul_committed['foul_committed'])
nested_df = nested_df.rename(columns={'card.name': 'card.name2',
                                      'penalty': 'penalty2',})
df_foul_committed = pd.concat([df_foul_committed[['player', 'card.name', 'penalty']], nested_df], axis=1)
df_foul_committed['card.name'] = df_foul_committed['card.name'].combine_first(df_foul_committed['card.name2'])
df_foul_committed['penalty'] = df_foul_committed['penalty'].combine_first(df_foul_committed['penalty2'])
df_foul_committed['yellow_card'] = df_foul_committed['card.name'] == 'Yellow Card'    
df_foul_committed['red_card'] = df_foul_committed['card.name'] == 'Red Card'
df_foul_committed = df_foul_committed.fillna(False)

df_foul_committed = df_foul_committed.groupby('player').agg(
    foul_committed=('player', 'size'),
    yellow_card=('yellow_card', 'sum'),
    red_card=('red_card', 'sum'),
    penalty_conceded=('penalty', 'sum')
)

df_stats_final = df_stats_final.merge(df_foul_committed, left_on='player_name', right_on='player', how='left')
df_stats_final['foul_commited_per_90_min'] = np.round(df_stats_final['foul_committed'] * 90 / df_stats_final['time'], 2)
df_stats_final



C:\Users\vtgra\AppData\Local\Temp\ipykernel_23928\4026576407.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_foul_committed = df_foul_committed.fillna(False)


,player_name,team_name,jersey_number,country.name,position,time,starting,subbing,win,draw,...,interception,interception_per_90_min,foul_won,penalty_won,foul_won_per_90_min,foul_committed,yellow_card,red_card,penalty_conceded,foul_commited_per_90_min
0,Abdoul Karim Yoda,Getafe,11,France,Left Back,1009.400000,8,10,2,6,...,17.0,1.52,10.0,0.0,0.89,16.0,2.0,0.0,0.0,1.43
1,Abdoulaye Doucouré,Granada,16,Mali,Left Defensive Midfield,1184.450000,13,2,5,4,...,17.0,1.29,8.0,0.0,0.61,20.0,3.0,0.0,0.0,1.52
2,Abraham González Casanova,Espanyol,10,Spain,Left Center Midfield,1128.083333,13,7,5,5,...,11.0,0.88,9.0,0.0,0.72,14.0,1.0,0.0,0.0,1.12
3,Adalberto Peñaranda Maestre,Granada,27,Venezuela (Bolivarian Republic),Right Center Forward,1694.933333,21,2,8,3,...,7.0,0.37,41.0,1.0,2.18,64.0,8.0,0.0,0.0,3.40
4,Aderllan Leandro de Jesus Santos,Valencia,4,Brazil,Left Center Back,1564.000000,17,0,5,3,...,25.0,1.44,6.0,0.0,0.35,13.0,1.0,0.0,1.0,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Íñigo Lekue Martínez,Athletic Club,30,Spain,Left Wing,1187.166667,12,8,10,5,...,15.0,1.14,10.0,0.0,0.76,13.0,1.0,0.0,0.0,0.99
410,Íñigo Martínez Berridi,Real Sociedad,6,Spain,Left Center Back,2760.000000,30,0,9,9,...,39.0,1.27,27.0,0.0,0.88,37.0,5.0,0.0,1.0,1.21
411,Óliver Torres Muñoz,Atlético Madrid,10,Spain,Right Midfield,838.916667,9,12,17,2,...,6.0,0.64,17.0,0.0,1.82,13.0,2.0,0.0,0.0,1.39
412,Óscar Esau Duarte Gaitán,Espanyol,24,Costa Rica,Left Center Back,1220.716667,13,1,6,3,...,15.0,1.11,5.0,0.0,0.37,23.0,5.0,0.0,0.0,1.70


## Ball recovery

In [20]:
df_ball_recovery = dfs_by_type_final['Ball Recovery'].drop(['out', 'offensive'], axis=1)
# Nested DataFrame dans la colonne 'ball_recovery'
def extract_recovery_failure(value):
    if isinstance(value, dict) and 'recovery_failure' in value:
        return value['recovery_failure']
    return np.nan

df_ball_recovery['recovery_failure2'] = df_ball_recovery['ball_recovery'].apply(extract_recovery_failure)
df_ball_recovery['recovery_failure'] = df_ball_recovery['recovery_failure'].combine_first(df_ball_recovery['recovery_failure2'])

# Ball recovery complète si pas de 'recovery_failure'
df_ball_recovery = df_ball_recovery[df_ball_recovery['recovery_failure'].isna()]
df_ball_recovery = df_ball_recovery.groupby('player').size().reset_index(name='ball_recovery')

df_stats_final = df_stats_final.merge(df_ball_recovery, left_on='player_name', right_on='player', how='left').drop('player', axis=1)
df_stats_final['ball_recovery_per_90_min'] = np.round(df_stats_final['ball_recovery'] * 90 / df_stats_final['time'], 2)
df_stats_final


,player_name,team_name,jersey_number,country.name,position,time,starting,subbing,win,draw,...,foul_won,penalty_won,foul_won_per_90_min,foul_committed,yellow_card,red_card,penalty_conceded,foul_commited_per_90_min,ball_recovery,ball_recovery_per_90_min
0,Abdoul Karim Yoda,Getafe,11,France,Left Back,1009.400000,8,10,2,6,...,10.0,0.0,0.89,16.0,2.0,0.0,0.0,1.43,64,5.71
1,Abdoulaye Doucouré,Granada,16,Mali,Left Defensive Midfield,1184.450000,13,2,5,4,...,8.0,0.0,0.61,20.0,3.0,0.0,0.0,1.52,88,6.69
2,Abraham González Casanova,Espanyol,10,Spain,Left Center Midfield,1128.083333,13,7,5,5,...,9.0,0.0,0.72,14.0,1.0,0.0,0.0,1.12,44,3.51
3,Adalberto Peñaranda Maestre,Granada,27,Venezuela (Bolivarian Republic),Right Center Forward,1694.933333,21,2,8,3,...,41.0,1.0,2.18,64.0,8.0,0.0,0.0,3.40,57,3.03
4,Aderllan Leandro de Jesus Santos,Valencia,4,Brazil,Left Center Back,1564.000000,17,0,5,3,...,6.0,0.0,0.35,13.0,1.0,0.0,1.0,0.75,48,2.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Íñigo Lekue Martínez,Athletic Club,30,Spain,Left Wing,1187.166667,12,8,10,5,...,10.0,0.0,0.76,13.0,1.0,0.0,0.0,0.99,64,4.85
410,Íñigo Martínez Berridi,Real Sociedad,6,Spain,Left Center Back,2760.000000,30,0,9,9,...,27.0,0.0,0.88,37.0,5.0,0.0,1.0,1.21,157,5.12
411,Óliver Torres Muñoz,Atlético Madrid,10,Spain,Right Midfield,838.916667,9,12,17,2,...,17.0,0.0,1.82,13.0,2.0,0.0,0.0,1.39,54,5.79
412,Óscar Esau Duarte Gaitán,Espanyol,24,Costa Rica,Left Center Back,1220.716667,13,1,6,3,...,5.0,0.0,0.37,23.0,5.0,0.0,0.0,1.70,52,3.83


## Block 

In [21]:
df_block = dfs_by_type_final['Block']
df_block = df_block.groupby('player').size().reset_index(name='block')

df_stats_final = df_stats_final.merge(df_block, left_on='player_name', right_on='player', how='left').drop('player', axis=1)
df_stats_final['block_per_90_min'] = np.round(df_stats_final['block'] * 90 / df_stats_final['time'], 2)
df_stats_final

,player_name,team_name,jersey_number,country.name,position,time,starting,subbing,win,draw,...,foul_won_per_90_min,foul_committed,yellow_card,red_card,penalty_conceded,foul_commited_per_90_min,ball_recovery,ball_recovery_per_90_min,block,block_per_90_min
0,Abdoul Karim Yoda,Getafe,11,France,Left Back,1009.400000,8,10,2,6,...,0.89,16.0,2.0,0.0,0.0,1.43,64,5.71,20.0,1.78
1,Abdoulaye Doucouré,Granada,16,Mali,Left Defensive Midfield,1184.450000,13,2,5,4,...,0.61,20.0,3.0,0.0,0.0,1.52,88,6.69,27.0,2.05
2,Abraham González Casanova,Espanyol,10,Spain,Left Center Midfield,1128.083333,13,7,5,5,...,0.72,14.0,1.0,0.0,0.0,1.12,44,3.51,24.0,1.91
3,Adalberto Peñaranda Maestre,Granada,27,Venezuela (Bolivarian Republic),Right Center Forward,1694.933333,21,2,8,3,...,2.18,64.0,8.0,0.0,0.0,3.40,57,3.03,40.0,2.12
4,Aderllan Leandro de Jesus Santos,Valencia,4,Brazil,Left Center Back,1564.000000,17,0,5,3,...,0.35,13.0,1.0,0.0,1.0,0.75,48,2.76,34.0,1.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Íñigo Lekue Martínez,Athletic Club,30,Spain,Left Wing,1187.166667,12,8,10,5,...,0.76,13.0,1.0,0.0,0.0,0.99,64,4.85,33.0,2.50
410,Íñigo Martínez Berridi,Real Sociedad,6,Spain,Left Center Back,2760.000000,30,0,9,9,...,0.88,37.0,5.0,0.0,1.0,1.21,157,5.12,47.0,1.53
411,Óliver Torres Muñoz,Atlético Madrid,10,Spain,Right Midfield,838.916667,9,12,17,2,...,1.82,13.0,2.0,0.0,0.0,1.39,54,5.79,31.0,3.33
412,Óscar Esau Duarte Gaitán,Espanyol,24,Costa Rica,Left Center Back,1220.716667,13,1,6,3,...,0.37,23.0,5.0,0.0,0.0,1.70,52,3.83,24.0,1.77


## Dribble 

In [22]:
df_dribble = dfs_by_type_final['Dribble']
df_dribble['complete'] = df_dribble['outcome.name'] == 'Complete'

df_dribble = df_dribble.groupby('player').agg(
    total_dribble=('player', 'size'),
    dribble=('complete', 'sum')
)
df_dribble['dribble_success_rate'] = np.round(df_dribble['dribble'] / df_dribble['total_dribble'] * 100, 2)
df_dribble = df_dribble.drop('total_dribble', axis=1)

df_stats_final = df_stats_final.merge(df_dribble, left_on='player_name', right_on='player', how='left')
df_stats_final['dribble_per_90_min'] = np.round(df_stats_final['dribble'] * 90 / df_stats_final['time'], 2)
df_stats_final


,player_name,team_name,jersey_number,country.name,position,time,starting,subbing,win,draw,...,red_card,penalty_conceded,foul_commited_per_90_min,ball_recovery,ball_recovery_per_90_min,block,block_per_90_min,dribble,dribble_success_rate,dribble_per_90_min
0,Abdoul Karim Yoda,Getafe,11,France,Left Back,1009.400000,8,10,2,6,...,0.0,0.0,1.43,64,5.71,20.0,1.78,40.0,59.70,3.57
1,Abdoulaye Doucouré,Granada,16,Mali,Left Defensive Midfield,1184.450000,13,2,5,4,...,0.0,0.0,1.52,88,6.69,27.0,2.05,10.0,55.56,0.76
2,Abraham González Casanova,Espanyol,10,Spain,Left Center Midfield,1128.083333,13,7,5,5,...,0.0,0.0,1.12,44,3.51,24.0,1.91,2.0,33.33,0.16
3,Adalberto Peñaranda Maestre,Granada,27,Venezuela (Bolivarian Republic),Right Center Forward,1694.933333,21,2,8,3,...,0.0,0.0,3.40,57,3.03,40.0,2.12,40.0,60.61,2.12
4,Aderllan Leandro de Jesus Santos,Valencia,4,Brazil,Left Center Back,1564.000000,17,0,5,3,...,0.0,1.0,0.75,48,2.76,34.0,1.96,8.0,88.89,0.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Íñigo Lekue Martínez,Athletic Club,30,Spain,Left Wing,1187.166667,12,8,10,5,...,0.0,0.0,0.99,64,4.85,33.0,2.50,18.0,39.13,1.36
410,Íñigo Martínez Berridi,Real Sociedad,6,Spain,Left Center Back,2760.000000,30,0,9,9,...,0.0,1.0,1.21,157,5.12,47.0,1.53,12.0,70.59,0.39
411,Óliver Torres Muñoz,Atlético Madrid,10,Spain,Right Midfield,838.916667,9,12,17,2,...,0.0,0.0,1.39,54,5.79,31.0,3.33,16.0,76.19,1.72
412,Óscar Esau Duarte Gaitán,Espanyol,24,Costa Rica,Left Center Back,1220.716667,13,1,6,3,...,0.0,0.0,1.70,52,3.83,24.0,1.77,2.0,100.00,0.15


## Dribbled past 

In [23]:
df_dribble_past = dfs_by_type_final['Dribbled Past']
df_dribble_past = df_dribble_past.groupby('player').size().reset_index(name='dribble_past')

df_stats_final = df_stats_final.merge(df_dribble_past, left_on='player_name', right_on='player', how='left').drop('player', axis=1)
df_stats_final['dribble_past_per_90_min'] = np.round(df_stats_final['dribble_past'] * 90 / df_stats_final['time'], 2)
df_stats_final

,player_name,team_name,jersey_number,country.name,position,time,starting,subbing,win,draw,...,foul_commited_per_90_min,ball_recovery,ball_recovery_per_90_min,block,block_per_90_min,dribble,dribble_success_rate,dribble_per_90_min,dribble_past,dribble_past_per_90_min
0,Abdoul Karim Yoda,Getafe,11,France,Left Back,1009.400000,8,10,2,6,...,1.43,64,5.71,20.0,1.78,40.0,59.70,3.57,25.0,2.23
1,Abdoulaye Doucouré,Granada,16,Mali,Left Defensive Midfield,1184.450000,13,2,5,4,...,1.52,88,6.69,27.0,2.05,10.0,55.56,0.76,16.0,1.22
2,Abraham González Casanova,Espanyol,10,Spain,Left Center Midfield,1128.083333,13,7,5,5,...,1.12,44,3.51,24.0,1.91,2.0,33.33,0.16,16.0,1.28
3,Adalberto Peñaranda Maestre,Granada,27,Venezuela (Bolivarian Republic),Right Center Forward,1694.933333,21,2,8,3,...,3.40,57,3.03,40.0,2.12,40.0,60.61,2.12,14.0,0.74
4,Aderllan Leandro de Jesus Santos,Valencia,4,Brazil,Left Center Back,1564.000000,17,0,5,3,...,0.75,48,2.76,34.0,1.96,8.0,88.89,0.46,12.0,0.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Íñigo Lekue Martínez,Athletic Club,30,Spain,Left Wing,1187.166667,12,8,10,5,...,0.99,64,4.85,33.0,2.50,18.0,39.13,1.36,14.0,1.06
410,Íñigo Martínez Berridi,Real Sociedad,6,Spain,Left Center Back,2760.000000,30,0,9,9,...,1.21,157,5.12,47.0,1.53,12.0,70.59,0.39,24.0,0.78
411,Óliver Torres Muñoz,Atlético Madrid,10,Spain,Right Midfield,838.916667,9,12,17,2,...,1.39,54,5.79,31.0,3.33,16.0,76.19,1.72,25.0,2.68
412,Óscar Esau Duarte Gaitán,Espanyol,24,Costa Rica,Left Center Back,1220.716667,13,1,6,3,...,1.70,52,3.83,24.0,1.77,2.0,100.00,0.15,10.0,0.74


## Carry

In [24]:
df_carry = dfs_by_type_final['Carry']
# Calculer la distance entre le début et la fin de chaque course
def calculate_distance(row):
    x1, y1 = row['location']
    x2, y2 = row['end_location']
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Appliquer la fonction à chaque ligne du DataFrame
df_carry['length'] = df_carry.apply(calculate_distance, axis=1)

df_carry = df_carry.groupby('player').agg(
    total_carry=('player', 'size'),
    mean_distance_carry=('length', 'mean')
)

df_stats_final = df_stats_final.merge(df_carry, left_on='player_name', right_on='player', how='left')
df_stats_final['carry_per_90_min'] = np.round(df_stats_final['total_carry'] * 90 / df_stats_final['time'], 2)
df_stats_final

,player_name,team_name,jersey_number,country.name,position,time,starting,subbing,win,draw,...,block,block_per_90_min,dribble,dribble_success_rate,dribble_per_90_min,dribble_past,dribble_past_per_90_min,total_carry,mean_distance_carry,carry_per_90_min
0,Abdoul Karim Yoda,Getafe,11,France,Left Back,1009.400000,8,10,2,6,...,20.0,1.78,40.0,59.70,3.57,25.0,2.23,447,8.225144,39.86
1,Abdoulaye Doucouré,Granada,16,Mali,Left Defensive Midfield,1184.450000,13,2,5,4,...,27.0,2.05,10.0,55.56,0.76,16.0,1.22,474,5.099006,36.02
2,Abraham González Casanova,Espanyol,10,Spain,Left Center Midfield,1128.083333,13,7,5,5,...,24.0,1.91,2.0,33.33,0.16,16.0,1.28,381,3.736005,30.40
3,Adalberto Peñaranda Maestre,Granada,27,Venezuela (Bolivarian Republic),Right Center Forward,1694.933333,21,2,8,3,...,40.0,2.12,40.0,60.61,2.12,14.0,0.74,510,9.051448,27.08
4,Aderllan Leandro de Jesus Santos,Valencia,4,Brazil,Left Center Back,1564.000000,17,0,5,3,...,34.0,1.96,8.0,88.89,0.46,12.0,0.69,460,4.619578,26.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Íñigo Lekue Martínez,Athletic Club,30,Spain,Left Wing,1187.166667,12,8,10,5,...,33.0,2.50,18.0,39.13,1.36,14.0,1.06,477,7.545894,36.16
410,Íñigo Martínez Berridi,Real Sociedad,6,Spain,Left Center Back,2760.000000,30,0,9,9,...,47.0,1.53,12.0,70.59,0.39,24.0,0.78,1005,8.702749,32.77
411,Óliver Torres Muñoz,Atlético Madrid,10,Spain,Right Midfield,838.916667,9,12,17,2,...,31.0,3.33,16.0,76.19,1.72,25.0,2.68,479,5.515401,51.39
412,Óscar Esau Duarte Gaitán,Espanyol,24,Costa Rica,Left Center Back,1220.716667,13,1,6,3,...,24.0,1.77,2.0,100.00,0.15,10.0,0.74,258,5.179060,19.02


## Duel

In [25]:
df_duel = dfs_by_type_final['Duel'].drop(['counterpress', 'type.id'], axis = 1)
df_duel['duel_won'] = df_duel['outcome.name'].isin(['Won', 'Success In Play', 'Success Out', 'Success'])

df_duel = df_duel.groupby('player').agg(
    total_duel=('player', 'size'),
    duel_won=('duel_won', 'sum')
)

df_duel['duel_won_rate'] = np.round(df_duel['duel_won'] / df_duel['total_duel'] * 100, 2)

df_stats_final = df_stats_final.merge(df_duel, left_on='player_name', right_on='player', how='left')
df_stats_final['duel_won_per_90_min'] = np.round(df_stats_final['duel_won'] * 90 / df_stats_final['time'], 2)
df_stats_final



,player_name,team_name,jersey_number,country.name,position,time,starting,subbing,win,draw,...,dribble_per_90_min,dribble_past,dribble_past_per_90_min,total_carry,mean_distance_carry,carry_per_90_min,total_duel,duel_won,duel_won_rate,duel_won_per_90_min
0,Abdoul Karim Yoda,Getafe,11,France,Left Back,1009.400000,8,10,2,6,...,3.57,25.0,2.23,447,8.225144,39.86,32.0,19.0,59.38,1.69
1,Abdoulaye Doucouré,Granada,16,Mali,Left Defensive Midfield,1184.450000,13,2,5,4,...,0.76,16.0,1.22,474,5.099006,36.02,53.0,14.0,26.42,1.06
2,Abraham González Casanova,Espanyol,10,Spain,Left Center Midfield,1128.083333,13,7,5,5,...,0.16,16.0,1.28,381,3.736005,30.40,46.0,16.0,34.78,1.28
3,Adalberto Peñaranda Maestre,Granada,27,Venezuela (Bolivarian Republic),Right Center Forward,1694.933333,21,2,8,3,...,2.12,14.0,0.74,510,9.051448,27.08,119.0,18.0,15.13,0.96
4,Aderllan Leandro de Jesus Santos,Valencia,4,Brazil,Left Center Back,1564.000000,17,0,5,3,...,0.46,12.0,0.69,460,4.619578,26.47,69.0,17.0,24.64,0.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Íñigo Lekue Martínez,Athletic Club,30,Spain,Left Wing,1187.166667,12,8,10,5,...,1.36,14.0,1.06,477,7.545894,36.16,68.0,28.0,41.18,2.12
410,Íñigo Martínez Berridi,Real Sociedad,6,Spain,Left Center Back,2760.000000,30,0,9,9,...,0.39,24.0,0.78,1005,8.702749,32.77,111.0,35.0,31.53,1.14
411,Óliver Torres Muñoz,Atlético Madrid,10,Spain,Right Midfield,838.916667,9,12,17,2,...,1.72,25.0,2.68,479,5.515401,51.39,41.0,12.0,29.27,1.29
412,Óscar Esau Duarte Gaitán,Espanyol,24,Costa Rica,Left Center Back,1220.716667,13,1,6,3,...,0.15,10.0,0.74,258,5.179060,19.02,43.0,8.0,18.60,0.59


## Shot 

In [26]:
df_shot = dfs_by_type_final['Shot'][['player', 'body_part.name', 'outcome.name']]
outcomes = df_shot['outcome.name'].unique()

# Créer une colonne booléenne pour chaque valeur
for outcome in outcomes:
    df_shot[outcome] = df_shot['outcome.name'] == outcome
df_shot =df_shot.groupby('player').agg(
    total_shot=('player', 'size'),
    goal=('Goal', 'sum'),
    saved=('Saved', 'sum'),
    blocked=('Blocked', 'sum'),
    post=('Post', 'sum'),
    shot_off_target = ('Off T', 'sum'),
)

df_shot['shot_on_target'] = df_shot['total_shot'] - df_shot['shot_off_target']

df_stats_final = df_stats_final.merge(df_shot, left_on='player_name', right_on='player', how='left')
df_stats_final['goal_per_90_min'] = np.round(df_stats_final['goal'] * 90 / df_stats_final['time'], 2)
df_stats_final['shot_per_90_min'] = np.round(df_stats_final['total_shot'] * 90 / df_stats_final['time'], 2)
df_stats_final['shot_on_target_per_90_min'] = np.round(df_stats_final['shot_on_target'] * 90 / df_stats_final['time'], 2)
df_stats_final['shots_on_target_rate'] = np.round(df_stats_final['shot_on_target'] / df_stats_final['total_shot'] * 100, 2)
df_stats_final['goal_conversion_rate'] = np.round(df_stats_final['goal'] / df_stats_final['total_shot'] * 100, 2)
df_stats_final


C:\Users\vtgra\AppData\Local\Temp\ipykernel_23928\4147042308.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shot[outcome] = df_shot['outcome.name'] == outcome
C:\Users\vtgra\AppData\Local\Temp\ipykernel_23928\4147042308.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shot[outcome] = df_shot['outcome.name'] == outcome
C:\Users\vtgra\AppData\Local\Temp\ipykernel_23928\4147042308.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

,player_name,team_name,jersey_number,country.name,position,time,starting,subbing,win,draw,...,saved,blocked,post,shot_off_target,shot_on_target,goal_per_90_min,shot_per_90_min,shot_on_target_per_90_min,shots_on_target_rate,goal_conversion_rate
0,Abdoul Karim Yoda,Getafe,11,France,Left Back,1009.400000,8,10,2,6,...,1.0,4.0,0.0,3.0,5.0,0.00,0.71,0.45,62.50,0.00
1,Abdoulaye Doucouré,Granada,16,Mali,Left Defensive Midfield,1184.450000,13,2,5,4,...,0.0,5.0,0.0,3.0,7.0,0.00,0.76,0.53,70.00,0.00
2,Abraham González Casanova,Espanyol,10,Spain,Left Center Midfield,1128.083333,13,7,5,5,...,7.0,3.0,1.0,4.0,12.0,0.08,1.28,0.96,75.00,6.25
3,Adalberto Peñaranda Maestre,Granada,27,Venezuela (Bolivarian Republic),Right Center Forward,1694.933333,21,2,8,3,...,3.0,6.0,0.0,8.0,14.0,0.27,1.17,0.74,63.64,22.73
4,Aderllan Leandro de Jesus Santos,Valencia,4,Brazil,Left Center Back,1564.000000,17,0,5,3,...,2.0,1.0,0.0,3.0,5.0,0.00,0.46,0.29,62.50,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Íñigo Lekue Martínez,Athletic Club,30,Spain,Left Wing,1187.166667,12,8,10,5,...,3.0,3.0,0.0,3.0,8.0,0.08,0.83,0.61,72.73,9.09
410,Íñigo Martínez Berridi,Real Sociedad,6,Spain,Left Center Back,2760.000000,30,0,9,9,...,3.0,6.0,0.0,5.0,11.0,0.03,0.52,0.36,68.75,6.25
411,Óliver Torres Muñoz,Atlético Madrid,10,Spain,Right Midfield,838.916667,9,12,17,2,...,2.0,3.0,0.0,4.0,7.0,0.00,1.18,0.75,63.64,0.00
412,Óscar Esau Duarte Gaitán,Espanyol,24,Costa Rica,Left Center Back,1220.716667,13,1,6,3,...,2.0,1.0,1.0,3.0,6.0,0.07,0.66,0.44,66.67,11.11


## Goalkeeper

In [27]:
df_goalkeeper_int = dfs_by_type_final['Goal Keeper'][['player', 'type.name']]
types = df_goalkeeper_int['type.name'].unique()

# Créer une colonne booléenne pour chaque valeur
for type in types:
    df_goalkeeper_int[type] = df_goalkeeper_int['type.name'] == type

df_goalkeeper_int = df_goalkeeper_int.groupby('player').agg(
    save=('Save', 'sum'),
    shot_saved=('Shot Saved', 'sum'),
    goal_conceded=('Goal Conceded', 'sum'),
    penalty_saved=('Penalty Saved', 'sum'),
    penalty_scored_against=('Penalty Conceded', 'sum'),
    punch=('Punch', 'sum'),
    keeper_sweeper=('Keeper Sweeper', 'sum'),
    smothers=('Smother', 'sum'),
    collected=('Collected', 'sum'),
)

df_goalkeeper_int['save'] = df_goalkeeper_int['save'] + df_goalkeeper_int['shot_saved'] + df_goalkeeper_int['penalty_saved']
df_goalkeeper_int['goal_conceded'] = df_goalkeeper_int['goal_conceded'] + df_goalkeeper_int['penalty_scored_against']
df_goalkeeper_int = df_goalkeeper_int.drop(['shot_saved'], axis=1)

df_stats_final = df_stats_final.merge(df_goalkeeper_int, left_on='player_name', right_on='player', how='left')
df_stats_final = df_stats_final.fillna(0)
df_stats_final['save_per_90_min'] = np.round(df_stats_final['save'] * 90 / df_stats_final['time'], 2)
df_stats_final['goal_conceded_per_90_min'] = np.round(df_stats_final['goal_conceded'] * 90 / df_stats_final['time'], 2)
df_stats_final['goal_conceded_rate'] = np.round(df_stats_final['goal_conceded'] / df_stats_final['save'] * 100, 2)
df_stats_final['penalty_saved_rate'] = np.where(
    (df_stats_final['penalty_scored_against'] + df_stats_final['penalty_saved']) != 0,
    np.round(df_stats_final['penalty_saved'] / (df_stats_final['penalty_scored_against'] + df_stats_final['penalty_saved']) * 100, 2),
    0
)
df_stats_final['punch_per_90_min'] = np.round(df_stats_final['punch'] * 90 / df_stats_final['time'], 2)
df_stats_final['keeper_sweeper_per_90_min'] = np.round(df_stats_final['keeper_sweeper'] * 90 / df_stats_final['time'], 2)
df_stats_final['smothers_per_90_min'] = np.round(df_stats_final['smothers'] * 90 / df_stats_final['time'], 2)
df_stats_final['collected_per_90_min'] = np.round(df_stats_final['collected'] * 90 / df_stats_final['time'], 2)

df_stats_final

C:\Users\vtgra\AppData\Local\Temp\ipykernel_23928\2014277105.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_goalkeeper_int[type] = df_goalkeeper_int['type.name'] == type
C:\Users\vtgra\AppData\Local\Temp\ipykernel_23928\2014277105.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_goalkeeper_int[type] = df_goalkeeper_int['type.name'] == type
C:\Users\vtgra\AppData\Local\Temp\ipykernel_23928\2014277105.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,player_name,team_name,jersey_number,country.name,position,time,starting,subbing,win,draw,...,smothers,collected,save_per_90_min,goal_conceded_per_90_min,goal_conceded_rate,penalty_saved_rate,punch_per_90_min,keeper_sweeper_per_90_min,smothers_per_90_min,collected_per_90_min
0,Abdoul Karim Yoda,Getafe,11,France,Left Back,1009.400000,8,10,2,6,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
1,Abdoulaye Doucouré,Granada,16,Mali,Left Defensive Midfield,1184.450000,13,2,5,4,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
2,Abraham González Casanova,Espanyol,10,Spain,Left Center Midfield,1128.083333,13,7,5,5,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
3,Adalberto Peñaranda Maestre,Granada,27,Venezuela (Bolivarian Republic),Right Center Forward,1694.933333,21,2,8,3,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
4,Aderllan Leandro de Jesus Santos,Valencia,4,Brazil,Left Center Back,1564.000000,17,0,5,3,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Íñigo Lekue Martínez,Athletic Club,30,Spain,Left Wing,1187.166667,12,8,10,5,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
410,Íñigo Martínez Berridi,Real Sociedad,6,Spain,Left Center Back,2760.000000,30,0,9,9,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
411,Óliver Torres Muñoz,Atlético Madrid,10,Spain,Right Midfield,838.916667,9,12,17,2,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0
412,Óscar Esau Duarte Gaitán,Espanyol,24,Costa Rica,Left Center Back,1220.716667,13,1,6,3,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0


## Player On/Player Off

In [28]:
# Informations déjà récupérées
del dfs_by_type_final['Player On']
del dfs_by_type_final['Player Off']

## Error 

In [29]:
# Manque de précision/d'explication sur la stat
del dfs_by_type_final['Error']

## Substitution

In [30]:
# Information déjà récupérée
del dfs_by_type_final['Substitution']

## Injury Stoppage

In [31]:
# Information inintéréssante
del dfs_by_type_final['Injury Stoppage']

# Shield

In [32]:
# Information sur la protection de balle pour sortie de but, peu intéressante ici 
del dfs_by_type_final['Shield']

In [33]:
del dfs_by_type_final['Own Goal Against']
del dfs_by_type_final['Own Goal For']

## Miscontrol 

In [34]:
# Inclus dans les balles perdues 
del dfs_by_type_final['Miscontrol']

## 50/50

In [35]:
# Inclus dans les duels
del dfs_by_type_final['50/50']

## Pressing

In [36]:
# Pas de précision sur la réussite du pressing, information non pertinente
del dfs_by_type_final['Pressure']

## Bad Behaviour

In [37]:
# Inclus dans les fautes 
del dfs_by_type_final['Bad Behaviour']

## Enregistrement de la BDD

In [38]:
df_stats_final = df_stats_final.replace({np.nan: 0})
# On renomme les colonnes
df_stats_final = df_stats_final.rename(columns={'complete_pass': 'Total Completed Passes', 'long_pass': 'Total Long Passes', 'medium_pass': 'Total Medium Passes', 'short_pass': 'Total Short Passes', 
                                                'goal_assist': 'Total Goal Assists', 'all_pass': 'Total Passes', 'shot_assist': 'Total Shot Assists', 'cross': 'Total Crosses', 
                                                'ball_receipt': 'Total Ball Receipts', 'lost_ball': 'Total Lost Balls', 'own_goal': 'Total Own Goals', 'dispossessed': 'Total Dispossessed', 'total_clearance': 'Total Clearances', 
                                                'left_foot_clearance': 'Total Left Foot Clearances', 'right_foot_clearance': 'Total Right Foot Clearances', 'head_clearance': 'Total Head Clearances', 'other_clearance': 'Total Other Clearances', 
                                                'aerial_won_clearance': 'Total Aerial Duels Won', 'interception': 'Total Interceptions', 'foul_won': 'Total Fouls Won', 'penalty_won': 'Total Penalties Won', 'foul_committed': 'Total Fouls Committed', 
                                                'yellow_card': 'Total Yellow Cards', 'red_card': 'Total Red Cards', 'penalty_conceded': 'Total Penalties Conceded', 'ball_recovery': 'Total Ball Recoveries', 'block': 'Total Blocks', 
                                                'dribble': 'Total Completed Dribbles', 'dribble_success_rate': 'Dribble Success Rate', 'dribble_past': 'Total Dribbled Past', 'total_carry': 'Total Carries', 
                                                'mean_distance_carry': 'Mean Distance Carries', 'total_duel': 'Total Duels', 'duel_won': 'Total Duels Won', 'duel_won_rate': 'Duel Won Rate', 'total_shot': 'Total Shots', 
                                                'goal': 'Total Goals', 'saved': 'Total Shots Saved', 'blocked': 'Total Shots Blocked', 'post': 'Total Shots On the Post', 'shot_off_target': 'Total Shots Off Target', 
                                                'shot_on_target': 'Total Shots On Target', 'shots_on_target_rate': 'Shots On Target Rate', 'goal_conversion_rate': 'Goal Conversion Rate', 'save': 'Total Saves', 
                                                'goal_conceded': 'Total Goals Conceded', 'penalty_saved': 'Total Penalties Saved', 'penalty_scored_against': 'Total Penalties Scored Against', 'punch': 'Total Punches', 
                                                'keeper_sweeper': 'Total Keeper Sweeper', 'smothers': 'Total Smothers', 'collected': 'Total Collected', 'complete_pass_per_90_min' : 'Completed Passes per 90 min',
                                                'long_pass_per_90_min': 'Long Passes per 90 min', 'medium_pass_per_90_min': 'Medium Passes per 90 min', 'short_pass_per_90_min': 'Short Passes per 90 min',
                                                'shot_assist_per_90_min': 'Shot Assists per 90 min', 'cross_per_90_min': 'Crosses per 90 min', 'pass_success_rate': 'Pass Success Rate', 'ball_receipt_per_90_min': 'Ball Receipts per 90 min',
                                                'lost_ball_per_90_min': 'Lost Balls per 90 min', 'clearance_per_90_min': 'Clearances per 90 min', 'aerial_duel_won_per_90_min': 'Aerial Duels Won per 90 min', 
                                                'interception_per_90_min': 'Interceptions per 90 min', 'foul_won_per_90_min': 'Fouls Won per 90 min', 'foul_commited_per_90_min': 'Fouls Committed per 90 min',
                                                'ball_recovery_per_90_min': 'Ball Recoveries per 90 min', 'block_per_90_min': 'Blocks per 90 min', 'dribble_per_90_min': 'Completed Dribbles per 90 min',
                                                'dribble_past_per_90_min': 'Dribbled Past per 90 min', 'carry_per_90_min': 'Carries per 90 min', 'duel_won_per_90_min': 'Duels Won per 90 min',
                                                'shot_per_90_min': 'Shots per 90 min', 'goal_per_90_min': 'Goals per 90 min', 'shot_on_target_per_90_min': 'Shots on Target per 90 min', 'goal_assist_per_90_min': 'Goal Assists per 90 min',
                                                'save_per_90_min': 'Saves per 90 min', 'goal_conceded_per_90_min': 'Goals Conceded per 90 min', 'goal_conceded_rate': 'Goals Conceded Rate', 'penalty_saved_rate': 'Penalties Saved Rate',
                                                'punch_per_90_min': 'Punch per 90 min','keeper_sweeper_per_90_min': 'Keeper Sweeper per 90 min' ,'smothers_per_90_min': 'Smothers per 90 min', 'collected_per_90_min': 'Collected per 90 min'})
df_stats_final.to_csv('data_app/stats_final.csv', index=False)

## Création d'un DataFrame avec les quantiles

In [39]:
df_stats_ranking = df_stats_final.copy()  
df_stats_ranking = df_stats_ranking.drop(['team_name','jersey_number', 'country.name', 'starting', 'subbing', 'win', 'draw', 'lose'], axis=1)
df_stats_ranking = df_stats_ranking[df_stats_ranking['position'] != 'Goalkeeper']

def compute_quantiles(df, descending_columns=[]):
    for column in df.select_dtypes(include=[np.number, bool]).columns:
        if column in descending_columns:
            df[column] = df[column].rank(ascending=False, pct=True) * 100
        else:
            df[column] = df[column].rank(ascending=True, pct=True) * 100
        
    return df


df_stats_ranking = compute_quantiles(
    df_stats_ranking, 
    descending_columns=['Total Lost Balls', 'Lost Balls per 90 min', 'Total Own Goals', 'Total Fouls Committed', 'Total Yellow Cards', 'Total Red Cards', 'Total Penalties Conceded', 'Fouls Committed per 90 min', 
    'Total Dribbled Past', 'Dribbled Past per 90 min', 'Total Goals Conceded', 'Total Penalties Scored Against', 'Goals Conceded per 90 min', 'Goals Conceded Rate']
)
df_stats_ranking.to_csv('data_app/stats_ranking.csv', index=False)

## Création de DataFrames avec les quantiles par position

In [40]:
descend_columns = ['Total Lost Balls', 'Lost Balls per 90 min', 'Total Own Goals', 'Total Fouls Committed', 'Yellow Cards', 'Red Cards', 'Total Penalties Conceded', 'Fouls Committed per 90 min', 
    'Total Dribbled Past', 'Dribbled Past per 90 min', 'Total Goals Conceded', 'Total Penalties Scored Against', 'Goals Conceded per 90 min', 'Goals Conceded Rate']

# Gardiens
df_stats_ranking_gk = df_stats_final[df_stats_final['position'] == 'Goalkeeper']
df_stats_ranking_gk = df_stats_ranking_gk.drop(['team_name','jersey_number', 'country.name', 'starting', 'subbing', 'win', 'draw', 'lose'], axis=1)

df_stats_ranking_gk = compute_quantiles(
    df_stats_ranking_gk,
    descending_columns=descend_columns
)
df_stats_ranking_gk.to_csv('data_app/stats_ranking_gk.csv', index=False)

# Latéraux 
df_stats_ranking_lb_rb = df_stats_final[(df_stats_final['position'] == 'Left Back') | (df_stats_final['position'] == 'Right Back')]
df_stats_ranking_lb_rb = df_stats_ranking_lb_rb.drop(['team_name','jersey_number', 'country.name', 'starting', 'subbing', 'win', 'draw', 'lose'], axis=1)

df_stats_ranking_lb_rb = compute_quantiles(
    df_stats_ranking_lb_rb, 
    descending_columns=descend_columns)
df_stats_ranking_lb_rb.to_csv('data_app/stats_ranking_lb_rb.csv', index=False)

# Centraux
df_stats_ranking_cb = df_stats_final[(df_stats_final['position'] == 'Left Center Back') | (df_stats_final['position'] == 'Right Center Back')]
df_stats_ranking_cb = df_stats_ranking_cb.drop(['team_name','jersey_number', 'country.name', 'starting', 'subbing', 'win', 'draw', 'lose'], axis=1)

df_stats_ranking_cb = compute_quantiles(
    df_stats_ranking_cb, 
    descending_columns=descend_columns
)
df_stats_ranking_cb.to_csv('data_app/stats_ranking_cb.csv', index=False)

# Milieux défensifs
df_stats_ranking_dm = df_stats_final[(df_stats_final['position'] == 'Left Defensive Midfield') | (df_stats_final['position'] == 'Right Defensive Midfield') | (df_stats_final['position'] == 'Center Defensive Midfield' )]
df_stats_ranking_dm = df_stats_ranking_dm.drop(['team_name','jersey_number', 'country.name', 'starting', 'subbing', 'win', 'draw', 'lose'], axis=1)

df_stats_ranking_dm = compute_quantiles(
    df_stats_ranking_dm, 
    descending_columns=descend_columns
)
df_stats_ranking_dm.to_csv('data_app/stats_ranking_dm.csv', index=False)

# Milieux centraux
df_stats_ranking_cm = df_stats_final[(df_stats_final['position'] == 'Left Center Midfield') | (df_stats_final['position'] == 'Right Center Midfield') | (df_stats_final['position'] == 'Left Midfield') | (df_stats_final['position'] == 'Right Midfield') | (df_stats_final['position'] == 'Center Attacking Midfield')]
df_stats_ranking_cm = df_stats_ranking_cm.drop(['team_name','jersey_number', 'country.name', 'starting', 'subbing', 'win', 'draw', 'lose'], axis=1)

df_stats_ranking_cm = compute_quantiles(
    df_stats_ranking_cm, 
    descending_columns=descend_columns
)
df_stats_ranking_cm.to_csv('data_app/stats_ranking_cm.csv', index=False)


# Ailiers
df_stats_ranking_w = df_stats_final[(df_stats_final['position'] == 'Left Wing') | (df_stats_final['position'] == 'Right Wing')]
df_stats_ranking_w = df_stats_ranking_w.drop(['team_name','jersey_number', 'country.name', 'starting', 'subbing', 'win', 'draw', 'lose'], axis=1)

df_stats_ranking_w = compute_quantiles(
    df_stats_ranking_w, 
    descending_columns=descend_columns
)  
df_stats_ranking_w.to_csv('data_app/stats_ranking_w.csv', index=False)

# Attaquants
df_stats_ranking_f = df_stats_final[(df_stats_final['position'] == 'Center Forward') | (df_stats_final['position'] == 'Right Center Forward') | (df_stats_final['position'] == 'Left Center Forward')]
df_stats_ranking_f = df_stats_ranking_f.drop(['team_name','jersey_number', 'country.name', 'starting', 'subbing', 'win', 'draw', 'lose'], axis=1)

df_stats_ranking_f = compute_quantiles(
    df_stats_ranking_f, 
    descending_columns=descend_columns
)
df_stats_ranking_f.to_csv('data_app/stats_ranking_f.csv', index=False)

